# Проект определения токсичности комментария

Проект оценки токсичности текста

Описание проекта
Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Задача - обучить модель классифицировать комментарии на позитивные и негативные.
Имеется набор данных с разметкой о токсичности правок.
Необходимо построить модель с целевым значением метрики качества F1 не ниже 0.75.

Данные находятся в файле toxic_comments.csv.
Столбец text в нём содержит текст комментария, а toxic — целевой признак.

In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.8 MB/s eta 0:00:00


In [3]:
!pip install spacy

In [4]:
from spacy.cli import download
download("en_core_web_trf")
download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
!pip install 'spacy[transformers]'

## Подготовка

In [34]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import re
import spacy_transformers
import spacy

from tqdm import tqdm

from google.colab import drive

**Посмотрим на исходные данные.**

In [8]:
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/projects/NLP_toxic_comments_classification/toxic_comments.csv')
data.head()

Mounted at /content/drive


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


**Удалим лишний столбец.**

In [9]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
data['toxic'].mean()

0.10161213369158527

**Токсических комментариев 10% от общего кол-ва в выборке.**

**Посмотрим на соотношение в абсолютных значениях.**

In [11]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

**Возьмем случайную выборку от исходного датасета, чтобы сохранить в ней аналогичное соотношение классов, как в исходной выборке и сделаем всю работу на ней, т.к. мощностей не хватает для работы на полном датасете, однако, если бы мощностей было достаточно, то работа производилась бы идентичная.**

In [12]:
df = data.sample(50000).reset_index(drop=True)
df['toxic'].mean()

0.10142

**Соотношение классов в новой выборке осталось аналогичным исходной.**

**Проведем подготовку данных, а именно сделаем токенизацию через инструменты библиотеки Spacy.**

In [21]:
nlp = spacy.load('en_core_web_sm')
tqdm.pandas()

new_corpus = []
for doc in tqdm(nlp.pipe(df['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(df['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))

df['lemm_text'] = new_corpus

100%|██████████| 50000/50000 [07:39<00:00, 108.91it/s]


**Очистим лемматизированный текст**

In [35]:
def clear_text(text):
  cleared_text = re.sub(r'[^a-zA-Z]', ' ',text)
  cleared_text = cleared_text.lower()
  return ' '.join(cleared_text.split())

df['clear_text'] = df['lemm_text'].progress_apply(clear_text)

100%|██████████| 50000/50000 [00:01<00:00, 25759.16it/s]


**Разобъем нашу выборку на тренировочную и тестовую. Валидационной не будет, т.к. будем использовать кросс-валидацию.**

In [37]:
#разбиваем признаки и целевые признаки на тренировочную и тестовую выборки в соотношении 80/20
feature = df[['clear_text']]
target = df['toxic']
train, test = train_test_split(df, test_size=0.2, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size = 0.2, random_state=1)

display('X_train shape:', X_train.shape)
display('X_test shape:', X_test.shape)
display('y_train shape:',y_train.shape)
display('y_test shape:',y_test.shape)

'X_train shape:'

(40000, 1)

'X_test shape:'

(10000, 1)

'y_train shape:'

(40000,)

'y_test shape:'

(10000,)

In [ ]:
cat_model = CatBoostClassifier(text_features=['clear_text'], loss_function='Logloss', task_type='CPU', random_seed=42, verbose=False)

parameters = {'learning_rate': [0.03],
              'depth': [6, 10],
              'l2_leaf_reg': [1, 3],
              'class_weights': [[1, 8.87]]
  }

cat_grid_search = GridSearchCV(cat_model, parameters, cv=3, n_jobs=1,  scoring='f1')
cat_grid_search.fit(X_train, y_train)

print(f"Best f1: {cat_grid_search.best_score_:.3f}, best params: {cat_grid_search.best_params_}")

**Создадим теперь корпус для тестовой и тренировочной выборок, применим TF_IDF векторизатор для получения фичей из исходного текста для тренировочной выборки, а затем на их основе создадим фичи и для тестовой выборки.**

In [28]:
%%time
corpus_train = train['lemm_text']
corpus_test = test['lemm_text']


count_tf_idf_train = TfidfVectorizer()

features_train = count_tf_idf_train.fit_transform(corpus_train)
target_train = train['toxic']
#features_test = count_tf_idf_train.transform(corpus_test)
#target_test = test['toxic']

CPU times: user 4.16 s, sys: 37.7 ms, total: 4.2 s
Wall time: 5.22 s


In [27]:
stopwords = list(spacy.lang.en.STOP_WORDS)

corpus_train = train['lemm_text']
corpus_test = test['lemm_text']
count_tf_idf_train = TfidfVectorizer()
features_train = count_tf_idf_train.fit_transform(corpus_train)
target_train = train['toxic']

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=1)),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': [1,2,6]
}

grid_search_tune = RandomizedSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)

best_f1_score = grid_search_tune.best_score_
print("Best F1 Score:", best_f1_score)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: ignored

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично, молодец, верно используешь Tfidf.
    
    
Совет: Внутри кросс-валидации происходит разбиение выборки на train и valid. Однако, в таком случае векторизатор обучен на всей выборке(train), а это не совсем корректно. Чтобы избежать это можно воспользоваться Pipeline:
    
```python
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=42)),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': [1,2,6]
}

grid_search_tune = RandomizedSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(train_features, train_targets)
  
    
```
    
Это просто каркас, можешь сам выбрать какие параметры использовать для подбора:)
    
+  https://runebook.dev/ru/docs/scikit_learn/modules/generated/sklearn.model_selection.halvinggridsearchcv - тут про HalvingGridSearchCV
    
+  https://www.rupython.com/python-sklearn-pipeline-pipeline-28301.html - про pipeline
+  https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d

</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Спасибо, про пайплайны почитаю, слышал о них, но ранее не использовал. По поводу обучения на всем трейне, в который включается валидационная часть - абсолютно согласен. Имеет ли смысл использовать тогда просто разбиение на отдельно трейн и валид выборки, если не хотим использовать пайплайн для получения более "честного" результата? </div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Имеешь ввиду, что если хочешь отказаться от pipeline, но при этом делать на три выборки, чтобы после кросс валидации еще и провалидироваться? Это не имеет смысла, так как валидационная выборка будет выступать в роли теста по сути (мы два раза проведем тест) + жалко данные(которые почти никак не используются) + кросс валидации гораздо более показательна, так как мы берем среднее
    
    
+  https://academy.yandex.ru/handbook/ml/article/kross-validaciya
+  https://academy.yandex.ru/handbook/ml/article/podbor-giperparametrov
</div>

## Обучение

### Модель классификатор решающего дерева

**Будем использовать GridSearchCV со встроенной кросс-валидацией для поиска наилучшего результата метрики f1, которую мы используем в качестве оценки нашей модели по условию задачи.**

In [29]:
%%time
model_tree = DecisionTreeClassifier(random_state=1)
param = {
         'criterion': ['gini', 'entropy'],
         'max_depth': range(1, 10, 3)
        }
gridsearch_tree = GridSearchCV(
    estimator=model_tree,
    param_grid=param,
    scoring='f1',
    cv=3)
gridsearch_tree.fit(features_train, target_train)
gridsearch_tree.best_params_

CPU times: user 59.6 s, sys: 217 ms, total: 59.8 s
Wall time: 1min


{'criterion': 'gini', 'max_depth': 7}

**Посмотрим теперь, какой наилучший результат метрики f1 удалось достигнуть данной моделью.**

In [30]:
f1_train_tree = round(gridsearch_tree.best_score_, 3)
f1_train_tree

0.559

### Модель классификатор CatBoost

**Будем использовать GridSearchCV со встроенной кросс-валидацией для поиска наилучшего результата метрики f1, которую мы используем в качестве оценки нашей модели по условию задачи. Выведем это значение метрики f1.**

In [32]:
%%time
model = CatBoostClassifier(silent=True, random_state=1)
param = {}
gridsearch_cat = GridSearchCV(
    estimator=model,
    param_grid=param,
    scoring='f1',
    cv=3)
gridsearch.fit(features_train, target_train)

KeyboardInterrupt: ignored

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Для Catboost будет лучше на самом деле воспользовать внутренним кодированием:
    
    
+  https://colab.research.google.com/drive/1bQOXIKpeyGZgda2qQqDijE2SIbSGX7-Z?usp=sharing    
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Имеешь в виду грузить сразу "сырой" текст в него? Или всё-таки необходимо его сначала очистить от спец-символов и стоп-слов?

Насколько кстати имеет смысл делать gridsearchCV для CatBoost'a с точки зрения подбора гиперпараметров? Слышал, что данная модель при жонглировании гиперпараметрами позволяет лишь чуть улучшить метрику, что справедливо для всяких кагловских соревнований только. Так ли это по твоему опыту?</div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Можно поэксперементировать, загрузить сырой или обработать
    
    
Да, в приницпе CatBoost работает из коробки уже хорошо, единственное стоит подобрать learning rate и кол-во итераций (чтобы модель точно сошлась)
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Можешь воспользоваться кросс валидацией cross_val_score</div>

## Анализ моделей

**Лучше из данных двух моделей проявила себя модель CatBoost. Проверим теперь, какой результат метрики f1 данная модель покажет на тестовой выборке.**

In [ ]:
%%time
predictions_test = gridsearch.best_estimator_.predict(features_test)
f1_test = f1_score(target_test, predictions_test)
f1_test

CPU times: user 188 ms, sys: 8.69 ms, total: 197 ms
Wall time: 77 ms


0.7565011820330969

**Удалось получить на тестовой выборке значение метрики f1 выше 0.75, что удовлетворяет исходной задаче.**

## Проверка моделей на адекватность

**Проверим модель на адекватность с помощью Dummy Model из класса sklearn, предварительно импортировав ее в первом пункте.**

**Проверка состоит в том, чтобы качество выбранной выше обученной модели было выше, чем качество модели Dummy, которая предсказывает результат не опираясь на признаки тренировочной выборки.**

In [ ]:
model_dummy = DummyClassifier(random_state=10)
parameters_dummy = {'strategy':['most_frequent', 'prior', 'stratified', 'uniform'],
                   }
gridsearch_dummy = GridSearchCV(
    estimator=model_dummy,
    param_grid=parameters_dummy,
    scoring='f1',
    cv=3, n_jobs=-1)
gridsearch_dummy.fit(features_train, target_train)
print('Лучшее значение метрики f1 для Dummy-модели на тренировочной выборке =', gridsearch_dummy.best_score_)

Лучшее значение метрики f1 для Dummy-модели на тренировочной выборке = 0.16992010542788896


In [ ]:
predict_dummy = gridsearch_dummy.best_estimator_.predict(features_test)
print('Лучшее значение метрики f1 для Dummy-модели на тестовой выборке =', f1_score(target_test, predict_dummy))

Лучшее значение метрики f1 для Dummy-модели на тестовой выборке = 0.15450785773366418


## Выводы

**В итоге с помощью Dummy модели удалось получить наилучшее качество модели с f1 равным 0.16 на тестовой выборке, что значительно ниже наилучшего результата выбранной и обученной модели CatBoost. Что доказывает адекватность найденной и выбранной нами модели.**

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>

Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множество реализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html
    
</div>